<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/unet_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setting environment variables

%env S_PATH=/content/sarpol-zahab-tents/data
%env U_PATH=/content/unet
%env D_PATH=/content/unet/data/tent
%env T_PATH=/content/unet/data/tent/train
%env I_PATH=/content/unet/data/tent/train/image
%env L_PATH=/content/unet/data/tent/train/label
%env A_PATH=/content/unet/data/tent/train/aug
%env V_PATH=/content/unet/data/tent/test
%env N_PATH=/content/unet/data/tent/npydata

env: S_PATH=/content/sarpol-zahab-tents/data
env: U_PATH=/content/unet
env: D_PATH=/content/unet/data/tent
env: T_PATH=/content/unet/data/tent/train
env: I_PATH=/content/unet/data/tent/train/image
env: L_PATH=/content/unet/data/tent/train/label
env: A_PATH=/content/unet/data/tent/train/aug
env: V_PATH=/content/unet/data/tent/test
env: N_PATH=/content/unet/data/tent/npydata


In [2]:
# Preparing directories
# TODO: make sure we pull the specific version of unet and the dataset so that if they change in the future we don't need to update the code


%%bash
if [ ! -d $S_PATH ]; then
 git clone https://github.com/tofighi/sarpol-zahab-tents.git
 git clone https://github.com/zhixuhao/unet.git
 
 rm -r $U_PATH/data
 rm -r $U_PATH/img
 
 mkdir -p $I_PATH
 mkdir -p $L_PATH
 mkdir -p $A_PATH
 
 mkdir -p $V_PATH
 mkdir -p $N_PATH

 rm -r /content/sample_data
fi

Cloning into 'sarpol-zahab-tents'...
Cloning into 'unet'...


In [3]:
# Image Processing

#From: https://medium.com/coinmonks/learn-how-to-train-u-net-on-your-dataset-8e3f89fbd623
#Fit the data into a 256*256 format and save it as a .tif file

from os import environ
from PIL import Image
from glob import glob

s_path = environ.get("S_PATH")
t_path = environ.get("T_PATH")
v_path = environ.get("V_PATH")

# TODO: make sure these lists are in the same order
imgs = list(map(Image.open, glob("{}/images/*".format(s_path))))
lbls = list(map(Image.open, glob("{}/labels/*".format(s_path))))

if len(imgs) == len(lbls):
  n = len(imgs)

  for i in range(n):
    # TODO: check that file names are the same
    # Any image modifications (cropping, etc) should be done here, before img.save()
    if i < n/2: # TODO: come up with a better way to divide the dataset
      imgs[i].save("{}/{}.png".format(v_path, i)) # we need to put some data in the test directory
    else:
      imgs[i].save("{}/image/{}.tif".format(t_path, i))
      lbls[i].save("{}/label/{}.tif".format(t_path, i))

In [4]:
# Modifying unet
%%bash
sed -i '8s/.*/from tensorflow.keras.optimizers import Adam/' unet/model.py
sed -i '55s/.*/    model = Model(inputs, conv10)/' unet/model.py
sed -i "s/membrane/tent/" unet/main.py


In [5]:
# Running dataPrepare.ipynb
%cd unet/

from data import *

#if you don't want to do data augmentation, set data_gen_args as an empty dict.
#data_gen_args = dict()

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGenerator = trainGenerator(20,'data/tent/train','image','label',data_gen_args,save_to_dir = "data/tent/train/aug")


#you will see 60 transformed images and their masks in data/membrane/train/aug
num_batch = 3
for i,batch in enumerate(myGenerator):
    if(i >= num_batch):
        break


image_arr,mask_arr = geneTrainNpy("data/tent/train/aug/","data/tent/train/aug/")
np.save("data/image_arr.npy",image_arr)
np.save("data/mask_arr.npy",mask_arr)

/content/unet
Found 128 images belonging to 1 classes.
Found 128 images belonging to 1 classes.


In [6]:
# Train Unet (trainUnet.ipynb)

from model import *
from data import *

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'data/tent/train','image','label',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_tent.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])
model.fit_generator(myGene,steps_per_epoch=2,epochs=5,callbacks=[model_checkpoint])

imgs_train,imgs_mask_train = geneTrainNpy("data/tent/train/aug/","data/tent/train/aug/")
model.fit(imgs_train, imgs_mask_train, batch_size=2, epochs=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

testGene = testGenerator("data/tent/test")
model = unet()
model.load_weights("unet_tent.hdf5")
results = model.predict(testGene,30,verbose=1)
saveResult("data/tent/test",results)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 128 images belonging to 1 classes.
Found 128 images belonging to 1 classes.
Epoch 1/5
2/2 [==============================] - 37s 466ms/step - loss: 0.2660 - accuracy: 0.9904

Epoch 00001: loss improved from inf to 0.26604, saving model to unet_tent.hdf5
Epoch 2/5
2/2 [==============================] - 1s 475ms/step - loss: 0.0444 - accuracy: 0.9984

Epoch 00002: loss improved from 0.26604 to 0.04443, saving model to unet_tent.hdf5
Epoch 3/5
2/2 [==============================] - 1s 488ms/step - loss: 0.0696 - accuracy: 0.9967

Epoch 00003: loss did not improve from 0.04443
Epoch 4/5
2/2 [==============================] - 1s 470ms/step - loss: 0.0740 - accuracy: 0.9955

Epoch 00004: loss did not improve from 0.04443
Epoch 5/5
2/2 [==============================] - 1s 463ms/step - loss: 0.0491 - accuracy: 0.9977

Epoch 00005: loss did not improve from 0.04443
Epoch 1/10
32/32 [==============================] - 19s 565ms/step - loss: 0.0338 - accuracy: 0.9975 - val_loss: 0.0319 - va

/content/unet/data.py:124: UserWarning: data/tent/test/0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/3_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/4_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/5_predict.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)
/content/unet/data.py:124: UserWarning: data/tent/test/6_predict.png is a low cont